# Sid's xA^TA

In [14]:
import numpy as np
import scipy.io as sio
from numpy.linalg import norm
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import BFGS

NID_An4 = sio.loadmat('180508_coordsFace1.mat')['thisFaceNodes_An4']
NC_An4 = sio.loadmat('180508_coordsFace1.mat')['triNodeCoords_An4'][:20]
NID_An5 = sio.loadmat('180508_coordsFace1.mat')['thisFaceNodes_An5']
NC_An5 = sio.loadmat('180508_coordsFace1.mat')['triNodeCoords_An5'][:20]


NC_An5[:,2] =  NC_An5[:,2] + 20;
# NC_An4 = np.array([[0,0,0], [0,1,0], [1,0,0], [1,1,0], [2,2,0]]);
# NC_An5 = np.array([[0,0,1], [0,1,1], [1,0,1], [1,1,1],[2,2,1]]);

m = NC_An4.shape[0];
n = NC_An5.shape[0];

x = np.repeat(NC_An4.reshape(m,1,3), n, axis=1);
y = np.repeat(NC_An5.reshape(n,1,3).swapaxes(0,1), m, axis=0);
if np.sum(x[:,0,:] == NC_An4) != m*3:
    print 'x shape wrong!'
elif np.sum(y[0,:,:] == NC_An5) != n*3:
    print 'y shape wrong!'
coeff = norm(x-y, axis=2)
coeff = np.reshape(coeff, m*n, )

bounds = Bounds(np.zeros((m*n, )), np.ones((m*n, ))*np.inf)

constraints = np.zeros((m*n, m+n))
for i in range(m):
    constraints[i*n : i*n+n, i] = 1
for i in range(n):
    constraints[np.arange(i, m*n, n), m+i] = 1
# print constrains[:, m+n-1].reshape(m,n)
lims = np.hstack((np.ones((m,)), np.ones((n,))*m/n))

linear_constraint = LinearConstraint(constraints.T, lims, lims)

weights_init = (np.ones((m,n))*1/n).reshape(m*n,)

obj = lambda weights: weights.T.dot(coeff)

# res = minimize(obj, weights_init, constraints=[linear_constraint], bounds=bounds, 
#                method='trust-constr', jac="2-point", hess=BFGS(), options={'gtol': 1e-4, 'disp': True})

def makeJac(x):
    return coeff
def makeHess(x):
    return np.zeros((coeff.shape[0], coeff.shape[0]))

res = minimize(obj, weights_init, constraints=[linear_constraint], bounds=bounds, 
               method='trust-constr', jac=makeJac, hess=makeHess, options={'gtol': 1e-3, 'disp': True})



The maximum number of function evaluations is exceeded.
Number of iterations: 1001, function evaluations: 1002, CG iterations: 1006, optimality: 2.44e-01, constraint violation: 3.16e-02, execution time: 9.3e+01 s.


In [65]:
weigths_init = np.ones((m,n))*1/n
print np.sum(weigths_init, axis=1).shape

print lims[0:459]

(5,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [9]:
res.x.reshape(m,n)

array([[0.52954748, 0.14295822, 0.14288233, 0.04849914, 0.04860376,
        0.02856102, 0.00924448, 0.00613993, 0.02849867, 0.01506498],
       [0.14212998, 0.24144812, 0.25068229, 0.10572103, 0.10609281,
        0.05280927, 0.0141505 , 0.0099445 , 0.05285748, 0.02416403],
       [0.14363892, 0.24762718, 0.2416856 , 0.10646414, 0.10636893,
        0.05286843, 0.01425743, 0.00995505, 0.05284462, 0.0242897 ],
       [0.04653659, 0.10970832, 0.10771679, 0.22603851, 0.22387585,
        0.10656416, 0.01732743, 0.01363039, 0.10621176, 0.0423902 ],
       [0.0466837 , 0.10924324, 0.10850042, 0.22403187, 0.2256367 ,
        0.10640884, 0.0171594 , 0.01361584, 0.10632411, 0.04239589],
       [0.02728758, 0.04678377, 0.04650388, 0.09814484, 0.09809557,
        0.23278825, 0.06022593, 0.03370658, 0.23105215, 0.12541144],
       [0.02730144, 0.04671954, 0.04653584, 0.09785153, 0.0980556 ,
        0.23112358, 0.06033183, 0.0337443 , 0.23364778, 0.12468856],
       [0.01363644, 0.02191126, 0.0218863

In [12]:
np.sum(res.x.reshape(m,n), axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])